<a href="https://colab.research.google.com/github/Simon-Pu/Temp/blob/master/CVPR2021_paper_statistics_using_chrome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import time

%matplotlib inline

In [ ]:
# Crawl the meta data from NeurIPS official homepage
# Set up a browser to crawl from dynamic web pages 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)



meta_list = [] 
wait_time = 0.5
max_try = 1000

title_list = []


for page_name in range(168, 180):
  # Load URL for all CVPR 2021 accepted papers.
  wd.get("http://cvpr2021.thecvf.com/node/{}".format(page_name)) #FIXME

  table =  wd.find_elements_by_tag_name("table") 
  print("length of table : ", len(table))

  for i in range(len(table)):
      tbody = table[i].find_elements_by_tag_name("tbody")
      trs = tbody[0].find_elements_by_tag_name("tr")
      
      for j in range(1, len(trs)):
          tds = trs[j].find_elements_by_tag_name('td')
          print(tds[1].text)
          title_list.append(tds[1].text)
          #print(i+1, "th table, ", j-1, "th paper : ", tds[3].text)
          #title_list.append(tds[3].text)

print("The number of total accepted paper titles : ", len(title_list))


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter

print(stopwords.words('english'))

stopwords_deep_learning = ['learning', 'network', 'neural', 'networks', 'deep', 'via', 'using', 'convolutional', 'single']

keyword_list = []

for i, title in enumerate(title_list):
  
  print(i, "th paper's title : ", title)
    
  word_list = title.split(" ")
  word_list = list(set(word_list))
    
  word_list_cleaned = [] 
  for word in word_list: 
    word = word.lower()
    if word not in stopwords.words('english') and word not in stopwords_deep_learning: #remove stopwords
          word_list_cleaned.append(word)  
    
  for k in range(len(word_list_cleaned)):
    keyword_list.append(word_list_cleaned[k])
  
keyword_counter = Counter(keyword_list)
print(keyword_counter)  

print('{} different keywords before merging'.format(len(keyword_counter)))

# Merge duplicates: CNNs and CNN
duplicates = []
for k in keyword_counter:
    if k+'s' in keyword_counter:
        duplicates.append(k)
for k in duplicates:
    keyword_counter[k] += keyword_counter[k+'s']
    del keyword_counter[k+'s']
print('{} different keywords after merging'.format(len(keyword_counter)))
print(keyword_counter)  

print("")

In [ ]:
# Show N most common keywords and their frequencies
num_keyowrd = 75 #FIXME
keywords_counter_vis = keyword_counter.most_common(num_keyowrd)

plt.rcdefaults()
fig, ax = plt.subplots(figsize=(8, 18))

key = [k[0] for k in keywords_counter_vis] 
value = [k[1] for k in keywords_counter_vis] 
y_pos = np.arange(len(key))
ax.barh(y_pos, value, align='center', color='green', ecolor='black', log=True)
ax.set_yticks(y_pos)
ax.set_yticklabels(key, rotation=0, fontsize=10)
ax.invert_yaxis() 
for i, v in enumerate(value):
    ax.text(v + 3, i + .25, str(v), color='black', fontsize=10)
ax.set_xlabel('Frequency')
ax.set_title('CVPR 2021 Submission Top {} Keywords'.format(num_keyowrd))

plt.show()

In [ ]:
# Show the word cloud forming by keywords
from wordcloud import WordCloud
wordcloud = WordCloud(max_font_size=128, max_words=160, 
                      width=1920, height=1080,
                      background_color="black").generate(' '.join(keyword_list))
plt.figure(figsize=(32, 16))
plt.imshow(wordcloud, interpolation="bilinear")
plt.title('CVPR 2021 Paper Keyword Statistics')
plt.axis("off")
plt.show()